In [6]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy import signal
import os
import glob
from matplotlib import rc
import io
import cv2
import datetime
from influxdb import InfluxDBClient # module needed to get access and send query to InfluxDB

rc("font", size=6, weight='bold')

# __Preprocessing Class__

In [12]:
class data_preprocessing():
    '''
    **Fileds**
    _user : 
    _passwd :
    _port :
    _addr :
    _dbname :
    _client :
    
    **Methods**
    
    '''
    def __init__(): # Not specified.
        self._user = 'user' 
        self._passwd = None
        self._data_dir = '.'
    def __init__(addr, port, username, passwd, dbname):
        self._user = username
        self._port = port
        self._passwd = passwd
        self._addr = addr
        self._dbname = dbname
        try:
            self._client = InfluxDBClient(host=self._addr, port=self._port,username=self._user, password=self._passwd,database=self._dbname)
        except Exception as error:
            print('Error occured while ' + repr(error))
        
    def __create_fft_plot__(fs, NFFT, var, time):
        '''
        Create fft heatmap and return plot axis.
        '''
        f_var, Pxx_den_var = signal.welch(var, fs, nperseg=len(x),
                                          scaling='density')  # Estimate Power Spectral Density using Welch’s method.
        mean_var = np.mean(10 * np.log10(Pxx_den_var))  # Compute mean of PSD in [dB].
        mean_varvar = [mean_var, mean_var]
        f_mean = [min(f_var), max(f_var)]

        my_x_ticks_1 = np.arange(0, 800, 200)  # Set axis-ticks.
        my_x_ticks_2 = np.arange(0, 800, 200)  # If use different dataset,
        my_x_ticks_3 = np.arange(-240, 0, 30)  # these may need to be changed.

        fig, (ax1) = plt.subplots(nrows=1, ncols=1)

        pxx, freq, bins, im = ax1.specgram(var, NFFT=NFFT, mode='psd', noverlap=0, scale='dB', Fs=fs, vmin=-125, vmax=-15,
                                           cmap='gray')  # Plot spectrogram.
        ax1.set_xlabel('Time [s]', fontweight='bold')
        ax1.set_ylabel('Frequency [Hz]', fontweight='bold')
        ax1.set_xticks(my_x_ticks_2)
        plt.xticks(fontsize=6)
        return ax1
    
    def __ax_to_ndarray__(ax, **kwargs):
        ax.axis('off') # remove axis
        ax.figure.canvas.draw() # first, draw image on canvas
        trans = ax.figure.dpi_scale_trans.inverted()
        bbox = ax.bbox.transformed(trans)

        buff = io.BytesIO() # send image to buffer, not file
        plt.savefig(buff, format='png', dpi=ax.figure.dpi, bbox_inches=bbox, **kwargs)
        buff.seek(0) # set file pointer to 0
        im = plt.imread(buff) # read it back, it returns ndarray class
        return im
    def save_images():
        '''
        Save FFT heatmap images for each device.
        '''
        return False
    '''
    Image Processing
    *****************************************************
    Connection to DB or I/O from DB
    '''
    def connect_to_InfluxDB(addr, port, username, passwd, dbname):
        '''
        This method is for connecting InfluxDB
        It needs 6 parameters, addr, port, username, passwd, dbname, and query.
        addr = IP address of Server which operates InfluxDB
        port = port number of InfluxDB
        username = username of InfluxDB
        passwd = password of InfluxDB
        dbname = database name
        '''
        try:
            self._client = InfluxDBClient(host=addr, port=port,username=username, password=passwd,database=dbname)
        except Exception as error:
            print('Error occured while ' + repr(error))
        
    def read_from_DB(query=None):
        '''
        This method is for reading data from InfluxDB described in self._client
        Need 1 parameter, query.
        
        query: query to send
        
        this method returns pandas DataFrame with columns=['dev_id','x','y','z', 'time']
        dev_id : device id
        x, y, z : acceleration value on each axis.
        time : epoch time(ms) when the acceleration value observed by device.
        '''
        def to_epoch(row):# str time to epoch
            unix_epch = datetime.datetime(1970,1,1)
            log_dt = datetime.datetime.strptime(row, "%Y-%m-%dT%H:%M:%S.%fZ")
            return int((log_dt -unix_epoch).total_seconds() * 1000)
        result = client.query(query)
        acc_dict_list = list()
        for data in result.get_points():
            acc_dict_list.append(data)
        acc_df = pd.DataFrame(acc_dict_list, columns=['dev_id', 'x','y','z','time'])
        acc_df['time'] = acc_df['time'].apply(to_epoch)
        
        return acc_df 
    
    def read_from_dir_csv(data_dir='.'):
        '''
        This method depict that object could get data from file system where python program is located.
        This method reads all .csv file in directory.
        **File type must be CSV.**
        
        It needs one parameter, data_dir.
        data_dir = Directory in which csv files are located. Default is current directory.
        '''
        qry = data_dir + '/*.csv'
        files = glob.glob(qry)
        
        
        return False # it will be substituted later.

## Test code

In [ ]:
def time_stft_psd_plot(fs, NFFT, var, time):
    f_var, Pxx_den_var = signal.welch(var, fs, nperseg=len(x),
                                      scaling='density')  # Estimate Power Spectral Density using Welch’s method.
    mean_var = np.mean(10 * np.log10(Pxx_den_var))  # Compute mean of PSD in [dB].
    mean_varvar = [mean_var, mean_var]
    f_mean = [min(f_var), max(f_var)]

    my_x_ticks_1 = np.arange(0, 800, 200)  # Set axis-ticks.
    my_x_ticks_2 = np.arange(0, 800, 200)  # If use different dataset,
    my_x_ticks_3 = np.arange(-240, 0, 30)  # these may need to be changed.

    fig, (ax1) = plt.subplots(nrows=1, ncols=1)

    pxx, freq, bins, im = ax1.specgram(var, NFFT=NFFT, mode='psd', noverlap=0, scale='dB', Fs=fs, vmin=-125, vmax=-15,
                                       cmap='gray')  # Plot spectrogram.
    ax1.set_xlabel('Time [s]', fontweight='bold')
    ax1.set_ylabel('Frequency [Hz]', fontweight='bold')
    ax1.set_xticks(my_x_ticks_2)
    plt.xticks(fontsize=6)
    return ax1

In [ ]:
def ax_to_ndarray(ax, **kwargs):

    ax.axis('off')
    ax.figure.canvas.draw()
    trans = ax.figure.dpi_scale_trans.inverted()
    bbox = ax.bbox.transformed(trans)

    buff = io.BytesIO()
    plt.savefig(buff, format='png', dpi=ax.figure.dpi, bbox_inches=bbox, **kwargs)
    buff.seek(0)
    im = plt.imread(buff)
    return im